In [ ]:
#Grę należy uruchomić w kompilatorze python
#Gra polega na utworzeniu superbohatera, który wykonuje misję i walczy z wrogrami. Bohater w czasie gry rozwija swoje atrybuty.
!pip install tkinter
!pip install sqlite3
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import sqlite3
from datetime import datetime
import threading
import time
import random


class DatabaseCreator:
    def __init__(self, db_name):
        self.db_name = db_name
        

    def create_database(self):
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()

        # Tworzenie tabeli "users", jeśli nie istnieje
        c.execute('''CREATE TABLE IF NOT EXISTS users
                     (username TEXT, password TEXT, superhero TEXT, life INTEGER,
                      defense INTEGER, strength INTEGER, level INTEGER, experience INTEGER, energy INTEGER)''')

        conn.commit()
        conn.close()

    def add_user(self, username, password, superhero, life, defense, strength, level, experience, energy):
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()

        # Sprawdzenie, czy dany username już istnieje
        c.execute("SELECT * FROM users WHERE username=?", (username,))
        existing_user = c.fetchone()

        if existing_user is None:
            # Dodanie nowego wpisu do tabeli
            c.execute("INSERT INTO users VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
                      (username, password, superhero, life, defense, strength, level, experience, energy))
            print("Użytkownik został dodany do bazy danych.")
        else:
            print("Użytkownik o podanym username już istnieje.")

        conn.commit()
        conn.close()

    def get_user(self, username):
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()

        # Pobranie informacji o użytkowniku na podstawie username
        c.execute("SELECT * FROM users WHERE username=?", (username,))
        user = c.fetchone()

        if user is not None:
            # Zwrócenie informacji o użytkowniku w postaci listy bez hasła
            user_info = list(user)
            # user_info.pop(1)  # Usunięcie hasła z listy
            return user_info
        else:
            print("Użytkownik o podanym username nie istnieje.")

        conn.close()

    def check_credentials(self, username, password):
        
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()

        # Sprawdzenie poprawności loginu i hasła
        c.execute("SELECT * FROM users WHERE username=? AND password=?", (username, password))
        user = c.fetchone()

        if user is not None:
            conn.close()
            return True
        else:
            conn.close()
            return False

    def update_user_superhero(self, username, superhero, life, defense, strength,energy):
        # Aktualizacja atrybutów superbohatera dla użytkownika w bazie danych
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()

        # Aktualizacja atrybutów superbohatera dla użytkownika
        query = "UPDATE users SET superhero=?, strength=?, defense=?, life=?,energy=? WHERE username=?"
        values = (superhero, strength, defense, life, energy,username)
        cursor.execute(query, values)

        # Zatwierdzenie zmian i zamknięcie połączenia z bazą danych
        conn.commit()
        conn.close()
        
    def update_user_energy(self, username, energy_change):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        c = conn.cursor()

        # Pobranie bieżącej wartości energii użytkownika
        c.execute("SELECT energy,experience,life,defense,strength,level FROM users WHERE username=?", (username,))
        current_energy = c.fetchone()

        if current_energy is not None:
            new_energy = current_energy[0] + energy_change
            experience = current_energy[1] + (5*energy_change)
            if new_energy >= 0:
                # Aktualizacja energii użytkownika
                if energy_change > 0:
                    if experience >= 100:
                        query = "UPDATE users SET energy=?,experience=?,life=?,defense=?,strength=?,level=? WHERE username=?"
                        values = (current_energy[0]+1, 0,current_energy[2] + 5,current_energy[3] + 5,current_energy[4] + 5,current_energy[5]+1,username)
                        
                    else:
                        query = "UPDATE users SET energy=?,experience=? WHERE username=?"
                        values = (new_energy, experience, username)
                else:
                    query = "UPDATE users SET energy=? WHERE username=?"
                    values = (new_energy, username)
                
                cursor.execute(query, values)
                conn.commit()
                conn.close()
                print("Energia użytkownika została zaktualizowana.")
            else:
                print("Operacja spowodowałaby ujemną wartość energii.")
        else:
            print("Użytkownik o podanym username nie istnieje.")

        conn.close()


class SuperHeroForm:
    def __init__(self, username, creator,root):
        self.username = username
        self.creator = creator
        self.main_window = root
        self.root = tk.Toplevel()
        self.root.title("SuperHero - Dodawanie Superbohatera")

        # Ustawienie stylu dla elementów interfejsu
        style = ttk.Style()
        style.configure("TLabel", font=("Arial", 12))
        style.configure("TButton", font=("Arial", 12))

        # Tworzenie etykiety informacyjnej
        self.label_info = ttk.Label(self.root, text="Dodawanie Superbohatera")
        self.label_info.pack(pady=20)

        # Tworzenie pól etykiet i wprowadzania danych
        self.label_superhero = ttk.Label(self.root, text="Nazwa Superbohatera:")
        self.label_superhero.pack()
        self.entry_superhero = ttk.Entry(self.root)
        self.entry_superhero.pack()

        self.label_strength = ttk.Label(self.root, text="Siła:")
        self.label_strength.pack()
        self.entry_strength = ttk.Entry(self.root)
        self.entry_strength.pack()

        self.label_defense = ttk.Label(self.root, text="Obrona:")
        self.label_defense.pack()
        self.entry_defense = ttk.Entry(self.root)
        self.entry_defense.pack()

        self.label_attack = ttk.Label(self.root, text="Życie:")
        self.label_attack.pack()
        self.entry_attack = ttk.Entry(self.root)
        self.entry_attack.pack()

        self.label_points = ttk.Label(self.root, text="Punkty dostępne: 10")
        self.label_points.pack(pady=10)

        # Tworzenie przycisku dodawania superbohatera
        self.button_add_superhero = ttk.Button(self.root, text="Dodaj Superbohatera", command=self.add_superhero)
        self.button_add_superhero.pack(pady=10)

    def add_superhero(self):
        superhero = self.entry_superhero.get()
        strength = int(self.entry_strength.get())
        defense = int(self.entry_defense.get())
        life = int(self.entry_attack.get())

        total_points = strength + defense + life

        if total_points <= 10:
            user_info = self.creator.get_user(self.username)

            if user_info[2] == "brak":
                self.creator.update_user_superhero(self.username, superhero, strength, defense, life,5)
                messagebox.showinfo("Sukces", "Superbohater został dodany.")
                self.root.destroy()
                self.main_window.destroy()
                game = SuperHeroGame(self.username)
                game.run()
            else:
                messagebox.showerror("Błąd", "Użytkownik posiada już przypisanego superbohatera.")
        else:
            messagebox.showerror("Błąd", "Suma punktów atrybutów przekracza 10.")
    

    def run(self):
        self.root.mainloop()
        
class TaskDatabase:
    def __init__(self, db_file,creator):
        self.db_file = db_file
        self.creator = creator
        
        
    def create_database(self):
        conn = sqlite3.connect(self.db_file)
        c = conn.cursor()

        c.execute('''CREATE TABLE IF NOT EXISTS tasks
                     (id INTEGER PRIMARY KEY AUTOINCREMENT,username text, task_number int, start_time real, end_time real,energy)''')

        conn.commit()
        conn.close()

    def add_task(self, username, task_number, start_time, end_time,energy):
        conn = sqlite3.connect(self.db_file)
        c = conn.cursor()
        last_insert_id = c.lastrowid
        c.execute("INSERT INTO tasks VALUES (?,?, ?, ?, ?,?)", (last_insert_id,username, task_number, start_time, end_time,energy))
        print("Dodałem zadanie")
       
        conn.commit()
        conn.close()
    
class Enemy:
    def __init__(self, name, strength, defense, life):
        self.name = name
        self.strength = strength
        self.defense = defense
        self.life = life

class SuperHeroGame:
    def __init__(self, username):
        self.username = username
        self.creator = DatabaseCreator("mydatabase.db")
        

        self.root = tk.Tk()
        self.root.title("SuperHero - Gra")
        self.root.geometry("800x600")

        # Ustawienie stylu dla elementów interfejsu
        style = ttk.Style()
        style.configure("TLabel", font=("Arial", 12))
        style.configure("TButton", font=("Arial", 12))

        # Tworzenie etykiety powitalnej
        self.label_welcome = ttk.Label(self.root, text="Witaj w grze SuperHero")
        self.label_welcome.pack(pady=20)

        user_info = self.creator.get_user(self.username)

        if user_info[2] == "brak":
            self.button_add_superhero = ttk.Button(self.root, text="Dodaj Superbohatera", command=self.open_superhero_form)
            self.button_add_superhero.pack()

        else:
         
            self.tree = ttk.Treeview(self.root,height=1)
            self.tree["columns"] = ("Superbohater", "Siła", "Obrona", "Życie","Poziom","Doświadczenie","Energia")
            self.tree.heading("#0", text="Nazwa użytkownika")
            self.tree.heading("Superbohater", text="Superbohater")
            self.tree.heading("Siła", text="Siła")
            self.tree.heading("Obrona", text="Obrona")
            self.tree.heading("Życie", text="Życie")
            self.tree.heading("Poziom", text="Poziom")
            self.tree.heading("Doświadczenie", text="Doświadczenie")
            self.tree.heading("Energia", text="Energia")
            self.tree.pack()

            # Dodanie danych użytkownika do tabeli
            self.tree.insert("", tk.END, text=user_info[0], values=(user_info[2], user_info[5], user_info[4], user_info[3],user_info[6],user_info[7],user_info[8]))
            self.tree.column("#0", width=100)  # Szerokość kolumny "#0" (Nazwa użytkownika)
            self.tree.column("Superbohater", width=100)  # Szerokość kolumny "Superbohater"
            self.tree.column("Siła", width=50)  # Szerokość kolumny "Siła"
            self.tree.column("Obrona", width=50)  # Szerokość kolumny "Obrona"
            self.tree.column("Życie", width=50)  # Szerokość kolumny "Życie"
            self.tree.column("Poziom", width=50)  # Szerokość kolumny "Siła"
            self.tree.column("Doświadczenie", width=100)  # Szerokość kolumny "Obrona"
            self.tree.column("Energia", width=50)  # Szerokość kolumny "Życie"
            
            self.button_tasks = ttk.Button(self.root, text="Zadania", command=self.open_task_window)
            self.button_tasks.pack()
            self.button_active_tasks = ttk.Button(self.root, text="Wyświetl aktywne zadania", command=self.open_active_tasks)
            self.button_active_tasks.pack()
            self.button_fight = tk.ttk.Button(self.root, text="Walka z wrogiem", command=self.fight_enemy)
            self.button_fight.pack()
            self.root.mainloop()
            
    def get_active_tasks(self, username):
        conn = sqlite3.connect("tasks.db")
        c = conn.cursor()
        analyzer = TaskAnalyzer("tasks.db", "mydatabase.db")
        analyzer.run()


        # Pobranie aktywnych zadań dla danego użytkownika
        c.execute("SELECT * FROM tasks WHERE username=?", (username,))
        tasks = c.fetchall()

        conn.close()
        
        
        return tasks
    def fight_enemy(self):
        enemy = Enemy("Evil Enemy", 10, 5, 100)  # Example enemy

        # Display enemy description window with an "Attack" button
        enemy_window = tk.Toplevel(self.root)
        enemy_window.title("Fight Enemy")
        enemy_window.geometry("400x300")

        # Create label with enemy description
        enemy_label = tk.Label(enemy_window, text=f"Enemy: {enemy.name}\n"
                                                   f"Strength: {enemy.strength}\n"
                                                   f"Defense: {enemy.defense}\n"
                                                   f"Life: {enemy.life}")
        enemy_label.pack(pady=20)

        # Create "Attack" button
        attack_button = tk.Button(enemy_window, text="Attack", command=lambda: self.attack_enemy(enemy_window, enemy))
        attack_button.pack()

    def attack_enemy(self, enemy_window, enemy):
        # Simulate fighting the enemy
        chance_to_win = 0.7  # Chance to win: 70%
        if random.random() < chance_to_win:
            messagebox.showinfo("Fight Enemy", "Victory!")
            experience = self.add_experience(20)  # Add 20 experience points on victory
            messagebox.showinfo("Add Experience", f"Added {experience} experience points!")
        else:
            messagebox.showinfo("Fight Enemy", "Defeat!")
            experience = self.add_experience(-40)
            messagebox.showinfo("Odejmij punkty", f"Added {experience} experience points!")

        enemy_window.destroy()
        self.root.destroy()
        game = SuperHeroGame(self.username)
        game.run()

    def add_experience(self, experience):
        # Add experience points to the user's profile
        # You should implement the appropriate database operations here
        conn = sqlite3.connect("mydatabase.db")
        cursor = conn.cursor()
        c = conn.cursor()

        # Pobranie bieżącej wartości energii użytkownika
        c.execute("SELECT energy,experience,life,defense,strength,level FROM users WHERE username=?", (self.username,))
        current_energy = c.fetchone()
        experience_current = current_energy[1]
        if experience_current >= 100:
            query = "UPDATE users SET energy=?,experience=?,life=?,defense=?,strength=?,level=? WHERE username=?"
            values = (current_energy[0] + 1,0,current_energy[2] + 5,current_energy[3] + 5,current_energy[4] + 5,current_energy[5] + 1,self.username)

        else:
            query = "UPDATE users SET experience=? WHERE username=?"
            values = (experience_current + experience, self.username)
        cursor.execute(query, values)
        conn.commit()
        conn.close()
        
        return experience


    def open_active_tasks(self):
        active_tasks = self.get_active_tasks(self.username)

        if active_tasks:
            message = r"Użytkownik\Numer zadania\Start\Koniec\Energia" + "\n\n"

            # Pobieranie danych z bazy
            for task in active_tasks:
                task_info = ""
                x = 0
                for column in task:
                    if (x >=3) and (x<5):
                        task_info += str(datetime.fromtimestamp(column).strftime("%Y-%m-%d %H:%M")) + "\t"  # Dodajemy kolumnę do stringa
                    else:
                        task_info += str(column) + "\t"  # Dodajemy kolumnę do stringa
                        
                    x += 1
                message += task_info + "\n"  # Dodajemy informacje o zadaniu do wiadomości

            messagebox.showinfo("Aktywne zadania", message)
            self.root.destroy()
            game = SuperHeroGame(self.username)
            game.run()
        else:
            messagebox.showinfo("Aktywne zadania", "Brak aktywnych zadań.")
        
    def create_table(self):
        user_info = self.creator.get_user(self.username)
        if user_info is not None:
            self.tree.delete(self.tree.get_children())
            # Tworzenie tabeli do wyświetlania danych użytkownika
            self.tree = ttk.Treeview(self.root,height=1)
            self.tree["columns"] = ("Superbohater", "Siła", "Obrona", "Życie","Poziom","Doświadczenie","Energia")
            self.tree.heading("#0", text="Nazwa użytkownika")
            self.tree.heading("Superbohater", text="Superbohater")
            self.tree.heading("Siła", text="Siła")
            self.tree.heading("Obrona", text="Obrona")
            self.tree.heading("Życie", text="Życie")
            self.tree.heading("Poziom", text="Poziom")
            self.tree.heading("Doświadczenie", text="Doświadczenie")
            self.tree.heading("Energia", text="Energia")
            self.tree.pack()

            # Dodanie danych użytkownika do tabeli
            self.tree.insert("", tk.END, text=user_info[0], values=(user_info[2], user_info[5], user_info[4], user_info[3],user_info[6],user_info[7],user_info[8]))
            self.tree.column("#0", width=100)  # Szerokość kolumny "#0" (Nazwa użytkownika)
            self.tree.column("Superbohater", width=100)  # Szerokość kolumny "Superbohater"
            self.tree.column("Siła", width=50)  # Szerokość kolumny "Siła"
            self.tree.column("Obrona", width=50)  # Szerokość kolumny "Obrona"
            self.tree.column("Życie", width=50)  # Szerokość kolumny "Życie"
            self.tree.column("Poziom", width=50)  # Szerokość kolumny "Siła"
            self.tree.column("Doświadczenie", width=100)  # Szerokość kolumny "Obrona"
            self.tree.column("Energia", width=50)  # Szerokość kolumny "Życie"

    def open_task_window(self):
        task_window = TaskWindow(self.username,self.root,self.creator)
        task_window.run()
        
    def open_superhero_form(self):
        superhero_form = SuperHeroForm(self.username, self.creator,self.root)
        superhero_form.run()

    def run(self):
        self.root.mainloop()

class TaskWindow:
    def __init__(self, username,root,creator):
        self.username = username
        self.root = root
        self.child_window = tk.Toplevel()
        self.creator = DatabaseCreator("mydatabase.db")
        self.child_window.title("SuperHero - Zadania")
        
        user_info = self.creator.get_user(self.username)
        self.tree = ttk.Treeview(self.child_window,height=1)
        self.tree["columns"] = ("Superbohater", "Siła", "Obrona", "Życie","Poziom","Doświadczenie","Energia")
        self.tree.heading("#0", text="Nazwa użytkownika")
        self.tree.heading("Superbohater", text="Superbohater")
        self.tree.heading("Siła", text="Siła")
        self.tree.heading("Obrona", text="Obrona")
        self.tree.heading("Życie", text="Życie")
        self.tree.heading("Poziom", text="Poziom")
        self.tree.heading("Doświadczenie", text="Doświadczenie")
        self.tree.heading("Energia", text="Energia")
        self.tree.pack()

        # Dodanie danych użytkownika do tabeli
        self.tree.insert("", tk.END, text=user_info[0], values=(user_info[2], user_info[5], user_info[4], user_info[3],user_info[6],user_info[7],user_info[8]))
        self.tree.column("#0", width=100)  # Szerokość kolumny "#0" (Nazwa użytkownika)
        self.tree.column("Superbohater", width=100)  # Szerokość kolumny "Superbohater"
        self.tree.column("Siła", width=50)  # Szerokość kolumny "Siła"
        self.tree.column("Obrona", width=50)  # Szerokość kolumny "Obrona"
        self.tree.column("Życie", width=50)  # Szerokość kolumny "Życie"
        self.tree.column("Poziom", width=50)  # Szerokość kolumny "Siła"
        self.tree.column("Doświadczenie", width=100)  # Szerokość kolumny "Obrona"
        self.tree.column("Energia", width=50)  # Szerokość kolumny "Życie"

        # Tworzenie tabeli do wyświetlania zadań
        self.tree = ttk.Treeview(self.child_window)
        self.tree["columns"] = ("Opis", "Czas trwania", "Koszt energii")
        self.tree.heading("#0", text="ID")
        self.tree.heading("Opis", text="Opis")
        self.tree.heading("Czas trwania", text="Czas trwania")
        self.tree.heading("Koszt energii", text="Koszt energii")
        self.tree.pack()

        # Inicjalizacja bazy danych
        self.task_db = TaskDatabase("tasks.db",self.creator)
        self.task_db.create_database()

        # Dodawanie zadań do tabeli
        self.add_tasks_to_table()

        # Przycisk "Wykonaj zadanie"
        self.button_execute = ttk.Button(self.child_window, text="Wykonaj zadanie", command=self.execute_task)
        self.button_execute.pack()

    def add_tasks_to_table(self):
        tasks = [
            {"opis": "Zadanie 1", "czas": "1 minuta", "energia": 1},
            {"opis": "Zadanie 2", "czas": "2 minuty", "energia": 2},
            {"opis": "Zadanie 3", "czas": "4 minuty", "energia": 4},
            {"opis": "Zadanie 4", "czas": "8 minut", "energia": 8},
            {"opis": "Zadanie 5", "czas": "16 minut", "energia": 16}
        ]

        for i, task in enumerate(tasks, start=1):
            self.tree.insert("", tk.END, text=str(i), values=(task["opis"], task["czas"], task["energia"]))

    def execute_task(self):
        selected_item = self.tree.selection()
        if selected_item:
            task_id = self.tree.item(selected_item)["text"]
            task_info = self.tree.item(selected_item)["values"]

            task_number = int(task_id)
            start_time = datetime.now().timestamp()
            end_time = start_time + int(task_info[1].split()[0]) * 60  # Przetwarzanie czasu trwania zadania w sekundach
            
            conn = sqlite3.connect("mydatabase.db")
            cursor = conn.cursor()
            c = conn.cursor()

            # Pobranie bieżącej wartości energii użytkownika
            c.execute("SELECT energy FROM users WHERE username=?", (self.username,))
            energy = c.fetchone()
            if energy[0] < task_info[2]:
                messagebox.showinfo("Błąd", "Za mało energii")
                
            else:
                self.task_db.add_task(self.username, task_number, start_time, end_time,task_info[2])
                self.creator.update_user_energy(self.username, -task_info[2])
            
            
            self.child_window.destroy()
            self.root.destroy()
            analyzer = TaskAnalyzer("tasks.db", "mydatabase.db")
            analyzer.run()
            game = SuperHeroGame(self.username)
            game.run()
            
            
            
            print(f"Zadanie {task_number} rozpoczęte o {datetime.fromtimestamp(start_time)} "
                  f"i zakończone o {datetime.fromtimestamp(end_time)}.")

            self.tree.delete(selected_item)

    def run(self):
        self.child_window.mainloop()




class LoginSystem:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("SuperHero - Logowanie")

        # Ustawienie stylu dla elementów interfejsu
        style = ttk.Style()
        style.configure("TLabel", font=("Arial", 12))
        style.configure("TButton", font=("Arial", 12))

        # Tworzenie etykiety informacyjnej
        self.label_info = ttk.Label(self.root, text="Logowanie do gry SuperHero")
        self.label_info.pack(pady=20)

        # Tworzenie pól etykiet i wprowadzania danych
        self.label_username = ttk.Label(self.root, text="Nazwa użytkownika:")
        self.label_username.pack()
        self.entry_username = ttk.Entry(self.root)
        self.entry_username.pack()

        self.label_password = ttk.Label(self.root, text="Hasło:")
        self.label_password.pack()
        self.entry_password = ttk.Entry(self.root, show="*")
        self.entry_password.pack()

        # Tworzenie przycisków logowania i rejestracji
        self.button_login = ttk.Button(self.root, text="Zaloguj się", command=self.login)
        self.button_login.pack(pady=10)
        self.button_register = ttk.Button(self.root, text="Zarejestruj się", command=self.register)
        self.button_register.pack()

        # Inicjalizacja obiektu DatabaseCreator
        self.creator = DatabaseCreator("mydatabase.db")

    def login(self):
        self.creator.create_database()
        username = self.entry_username.get()
        password = self.entry_password.get()

        if self.creator.check_credentials(username, password):
            messagebox.showinfo("Sukces", "Poprawne dane logowania.")
            self.root.destroy()
            game = SuperHeroGame(username)
            game.run()
            print("Uruchamiam analize zadan")
            
            
            
        else:
            messagebox.showerror("Błąd", "Niepoprawna nazwa użytkownika lub hasło.")

    def register(self):
        self.creator.create_database()
        username = self.entry_username.get()
        password = self.entry_password.get()

        if username and password:
            
            self.creator.add_user(username, password, "brak", 0, 0, 0, 0, 0, 0)
            messagebox.showinfo("Sukces", "Użytkownik został zarejestrowany.")
        else:
            messagebox.showerror("Błąd", "Wprowadź nazwę użytkownika i hasło.")

    def run(self):
        self.root.mainloop()
        
class TaskAnalyzer:
    def __init__(self, task_db_file, user_db_file):
        self.task_db_file = task_db_file
        self.user_db_file = user_db_file
        self.creator = DatabaseCreator("mydatabase.db")
        self.task_db = TaskDatabase("tasks.db",self.creator)
        self.task_db.create_database()
        
        

    def analyze_tasks(self):
        conn = sqlite3.connect(self.task_db_file)
        c = conn.cursor()

        # Pobranie zakończonych zadań
        c.execute("SELECT * FROM tasks")
        completed_tasks = c.fetchall()

        conn.close()

        # Sprawdzenie, czy zadania zostały zakończone
        if completed_tasks:
            for x in completed_tasks:
                start_time = datetime.now().timestamp()
                # Pobranie nazwy użytkownika na podstawie zakończonego zadania (może być tylko jedno)
                if start_time > x[4]:
                    username = x[1]
                    energia = x[2]

                    print("Zwróciłem energie")
                    # Zwrócenie energii użytkownikowi
                    user_db = DatabaseCreator(self.user_db_file)
                    user_db.update_user_energy(username, energia)  # Przykładowa wartość energii

                    # Usunięcie zakończonych zadań z bazy danych
                    conn = sqlite3.connect(self.task_db_file)
                    c = conn.cursor()
                    print(x[0],x[1])
                    c.execute("DELETE FROM tasks WHERE id=? AND username=? ", (x[0],x[1]))
                    conn.commit()
                    conn.close()
        
    def run(self):
        self.analyze_tasks()
           


def is_database_locked(database_path):
    try:
        connection = sqlite3.connect(database_path)
        connection.execute('SELECT 1')
        connection.close()
        return False
    except sqlite3.OperationalError as error:
        if 'database is locked' in str(error):
            return True
        else:
            raise
            
# Inicjalizacja systemu logowania
def run_login_system():
    database_path1 = "tasks.db"
    database_path2 = "mydatabase.db"
    if is_database_locked(database_path1) and is_database_locked(database_path2) :
        print("zajeta")
    else:
        login_system = LoginSystem()
        login_system.run()



            
def run_task_analyzer():
    # Przykładowe użycie

    database_path1 = "tasks.db"
    database_path2 = "mydatabase.db"
    print(is_database_locked(database_path1),is_database_locked(database_path2))
    if is_database_locked(database_path1) and is_database_locked(database_path2) :
        print("Baza danych jest zablokowana.")
    else:
        print("Baza danych jest dostępna.")
        analyzer = TaskAnalyzer("tasks.db", "mydatabase.db")
        analyzer.run()

# Tworzenie wątków dla funkcji
thread_login_system = threading.Thread(target=run_login_system)
# Uruchamianie wątków
thread_login_system.start()
# Oczekiwanie na zakończenie wątków
thread_login_system.join()